In [44]:
import sys

import itertools
import numpy as np

import itertools
import tqdm
import pandas as pd

import dask

import pandas as pd

from importlib.machinery import SourceFileLoader

main = SourceFileLoader("main", "../src/main.py").load_module()

from main import invariant_distribution, calculate_M, invariant_distribution_analytically

In [45]:
def match_payoff(player, coplayer, Sx):
    
    M = calculate_M(player, coplayer, analytical=False)
    
    ss = invariant_distribution(M)

    return ss @ Sx

In [46]:
M = calculate_M(strategy, deterministic_strategies[0], analytical=False)

M

array([[0.     , 0.54881, 0.     , 0.45119],
       [0.     , 0.71519, 0.     , 0.28481],
       [0.     , 0.60276, 0.     , 0.39724],
       [0.     , 0.54488, 0.     , 0.45512]])

In [53]:
def task(i, strategy, coplayers, labels, Sx, b, c):

    sx = match_payoff(strategy, strategy, Sx)
    
    data = []

    for label, coplayer in zip(labels, coplayers):

        sy = match_payoff(coplayer, strategy, Sx)
        close = np.isclose(sx, sy, atol=10 ** -4)
        best_response = sx < sy

        data_point = [
            i,
            *strategy,
            label,
            sx,
            sy,
            close,
            best_response,
            b,
            c,
        ]
        data.append(data_point)

    return data

In [72]:
max_simulation_number = 10 ** 7
b = 2
c = 1
n = 5
seed = 0

deterministic_strategies = list(
    itertools.product([0, 1], repeat=4)
)

labels = [f"N{i}" for i, _ in enumerate(deterministic_strategies)]

Sx = np.array([b - c, -c, b, 0])

results = []

for i in tqdm.notebook.tqdm(range(max_simulation_number)):
    
    np.random.seed(i)
    
    strategy = np.random.random((1, 4)).round(5)[0]
    
    data = task(i, strategy, deterministic_strategies, labels, Sx, b, c)
    
    results.append(data)

In [73]:
results = [item for sublist in results for item in sublist]

In [74]:
df = pd.DataFrame(results)

In [75]:
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0.54881,0.71519,0.60276,0.54488,N0,0.600676,1.313454,False,True,2,1
1,0,0.54881,0.71519,0.60276,0.54488,N1,0.600676,1.025642,False,True,2,1
2,0,0.54881,0.71519,0.60276,0.54488,N2,0.600676,0.868038,False,True,2,1
3,0,0.54881,0.71519,0.60276,0.54488,N3,0.600676,0.709784,False,True,2,1
4,0,0.54881,0.71519,0.60276,0.54488,N4,0.600676,1.313454,False,True,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
159995,9999,0.82339,0.21860,0.03146,0.54593,N11,0.491050,0.416419,False,False,2,1
159996,9999,0.82339,0.21860,0.03146,0.54593,N12,0.491050,-0.697602,False,False,2,1
159997,9999,0.82339,0.21860,0.03146,0.54593,N13,0.491050,-0.697602,False,False,2,1
159998,9999,0.82339,0.21860,0.03146,0.54593,N14,0.491050,-0.697602,False,False,2,1


In [116]:
bs = range(1, 5)

cs = range(1, 11)

In [192]:
to_search = np.zeros((len(bs), len(cs)))

for i, b_val in enumerate(bs):
    for j, c_val in enumerate(cs):
        to_search[i, j] = c_val / b_val
        

In [193]:
import sympy as sym

In [198]:
to_search[0]

array([0.5 , 0.75, 1.  , 1.25])

In [195]:
3 * 0.5

1.5

In [196]:
cs = np.arange(1, 3, .5)

bs = np.arange(2, 6, 1)

In [203]:
c = 2.5

b = 2

In [204]:
np.isclose(c / b, 1.25)

True

In [205]:
c / b

1.25